# Critical Scale Invariance in a Healthy Human Heart Rate

### Description

In this project you will analyse from a physics perspective time series of human heart rate. You will demonstrate the robust scale-invariance in the probability density function (PDF) of detrended healthy human heart rate increments. Moreover, you will show that such increments are not Gaussian distributed, but they display fat tails. This scale-independent and fractal structure supports the view that a healthy human heart rate is controlled to converge continually to a critical state.

### References

* Original paper: https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.93.178103


### Assignment

1. build the cumulative time series B(i) from the detrended and normalised heart beat time series b(i)
2. Do a polynomial fit of B(i)
3. Calculate the  increments (fluctuation) from the polynomial fit
4. Build the increments PDF and fit it with Gaussian and non Gaussian distributions.
5. Test the scale invariance of the PDFs by collapse plot



### Contacts

Samir Suweis <samir.suweis@unipd.it>